#credit to pascal voc 2008 data set

@misc{pascal-voc-2008,
	author = "Everingham, M. and Van~Gool, L. and Williams, C. K. I. and Winn, J. and Zisserman, A.",
	title = "The {PASCAL} {V}isual {O}bject {C}lasses {C}hallenge 2008 {(VOC2008)} {R}esults",
	howpublished = "http://www.pascal-network.org/challenges/VOC/voc2008/workshop/index.html"}

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


#### Import the data from the folders created to hold the original images and their corresponding image with a watermark. Standardize and reshape the images to fit into the network. 

In [2]:
raw_dir = '/gpfs_home/guest321/FinalProj/FinalProjectFinal/RAWS/'
wat_dir = '/gpfs_home/guest321/FinalProj/FinalProjectFinal/WATS/'

raw_datagen = ImageDataGenerator(rescale=1./255)

raw_generator = raw_datagen.flow_from_directory(
        raw_dir,  # this is the target directory
        target_size=(250, 250),
        batch_size=5096, shuffle = 0,
        class_mode="categorical")

wat_datagen = ImageDataGenerator(rescale=1./255)

wat_generator = wat_datagen.flow_from_directory(
        wat_dir,  # this is the target directory
        target_size=(250, 250),
        batch_size=5096, shuffle = 0,
        class_mode="categorical")

x_raw,y = raw_generator.next()
x_wat,y = wat_generator.next()

Found 5096 images belonging to 1 classes.
Found 5096 images belonging to 1 classes.


#### Split the data into training and validation sets. 

In [3]:
x_train_raw = x_raw[0:4500]
x_train_wat = x_wat[0:4500]

x_val_raw = x_raw[4500:4900]
x_val_wat = x_wat[4500:4900]

x_train = np.vstack((x_train_raw, x_train_wat))
x_val = np.vstack((x_val_raw, x_val_wat))

In [4]:
y_train = np.repeat([0.], 4500)
y_train = np.append(y_train, np.repeat([1.], 4500))

y_val = np.repeat([0.], 400)
y_val = np.append(y_val, np.repeat([1.], 400))

#### Create the response variables for each data set: images with a watermark have a response of 1, while images without a watermark have a response of 0. 

In [6]:
model = keras.models.Sequential()
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='elu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='elu'))
model.add(layers.Conv2D(100, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((4, 4)))

model.add(layers.Conv2D(75, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(202, activation='elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(40, activation='elu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['acc'])
#model.summary()
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 30, 
          batch_size = 40, verbose = 0)

Train on 9000 samples, validate on 800 samples
Epoch 1/30
9000/9000 [==============================] - 26s 3ms/sample - loss: 0.7138 - acc: 0.5132 - val_loss: 0.6797 - val_acc: 0.5550
Epoch 2/30
9000/9000 [==============================] - 19s 2ms/sample - loss: 0.6238 - acc: 0.6436 - val_loss: 0.4967 - val_acc: 0.7625
Epoch 3/30
9000/9000 [==============================] - 19s 2ms/sample - loss: 0.5295 - acc: 0.7329 - val_loss: 0.4411 - val_acc: 0.7987
Epoch 4/30
9000/9000 [==============================] - 19s 2ms/sample - loss: 0.4701 - acc: 0.7747 - val_loss: 0.4204 - val_acc: 0.7962
Epoch 5/30
9000/9000 [==============================] - 19s 2ms/sample - loss: 0.4432 - acc: 0.7971 - val_loss: 0.3887 - val_acc: 0.8350
Epoch 6/30
9000/9000 [==============================] - 19s 2ms/sample - loss: 0.3938 - acc: 0.8183 - val_loss: 0.3904 - val_acc: 0.8300
Epoch 7/30
9000/9000 [==============================] - 19s 2ms/sample - loss: 0.3801 - acc: 0.8312 - val_loss: 0.4616 - val_acc: 0

In [7]:
#write the history to a csv for use in plotting

df = pd.DataFrame(history.history)
df.to_csv('history.csv')